In [1]:
from similarity import similarity

# Computes the intra and inter semantic similarity between two neurons
# in: n1@str, n2@str: the two neurons to measure similarity
# out: (score:float, [(matching_properties, explanation@str)])
#similarity('PV Layer 1 cortex acetylcholine neuron', 'Layer 1a striatum gaba fast-spiking neuron')
similarity('Periventricular zone neuron', 'Hypothalamic medial zone neuron')

['157', '1097']
['467', '1097']
['1097']
('s_intra', (1.0, [[], (['ABA_REGION:1097'], 'located on same region')]), 's_inter', (0, []))


(1.0, [[], (['ABA_REGION:1097'], 'located on same region')])

In [ ]:
import pandas as pd


# download neuroelectro summary spreadsheet from URL and import into pandas data frame
neuroelectro_dataset_url = "http://dev.neuroelectro.org/static/src/article_ephys_metadata_curated.csv"
table = pd.read_csv(neuroelectro_dataset_url, sep='\t', index_col = 0, header=0)

In [2]:
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache


In [5]:
mcc = MouseConnectivityCache()
onto = mcc.get_ontology()
dataframe = onto.df

In [7]:
dataframe.id

id
997                997
8                    8
567                567
688                688
695                695
315                315
184                184
68                  68
667                667
500                500
107                107
219                219
299                299
644                644
947                947
985                985
320                320
943                943
648                648
844                844
882                882
993                993
656                656
962                962
767                767
1021              1021
1085              1085
453                453
12993            12993
12994            12994
               ...    
116                116
124                124
129                129
140                140
145                145
153                153
164                164
1024              1024
1032              1032
1055              1055
1063              1063
1071              1071
1078    